(mpi-point2point)=
# 点对点通信

一个最简单的通信模式点对点（Point-to-Point）通信，点对点通信又分为阻塞式（Blocking）和非阻塞式（Non-Blocking）。实现点对点时主要考虑两个问题：

* 如何控制和识别不同的进程？比如，想让 Rank 为 0 的进程给 Rank 为 1 的进程发消息。
* 如何控制数据的读写？多大的数据，数据类型是什么？

## 发送与接收

[`Comm.send`](https://mpi4py.readthedocs.io/en/latest/reference/mpi4py.MPI.Comm.html#mpi4py.MPI.Comm.send) 和 [`Comm.recv`](https://mpi4py.readthedocs.io/en/latest/reference/mpi4py.MPI.Comm.html#mpi4py.MPI.Comm.recv) 分别用来阻塞式地发送和接收数据。

`Comm.send(obj, dest, tag=0)` 的参数主要是 `obj` 和 `dest`。`obj` 就是我们想要发送的数据，数据可以是 Python 内置的数据类型，比如 `list` 和 `dict` 等，也可以是 NumPy 的 `ndarray`，甚至是 GPU 上的 cupy 数据。上一节 {ref}`mpi-hello-world` 我们介绍了 Communicator 和 Rank，可以通过 Rank 的号码来定位一个进程。`dest` 可以用 Rank 号码来表示。`tag` 主要用来标识，给程序员一个精细控制的选项，使用 `tag` 可以实现消息的有序传递和筛选。接收方可以选择只接收特定标签的消息，或者按照标签的顺序接收消息，以便更加灵活地控制消息的发送和接收过程。

## 案例1：发送 Python 对象

比如，我们发送一个 Python 对象。Python 对象在通信过程中的序列化使用的是 [pickle](https://docs.python.org/3/library/pickle.html#module-pickle)。

```python
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

if rank == 0:
    data = {'a': 7, 'b': 3.14}
    comm.send(data, dest=1)
    print(f"Sended: {data}, from rank: {rank}.")
elif rank == 1:
    data = comm.recv(source=0)
    print(f"Received: {data}, to rank: {rank}.")
```

在命令行中这样启动：

```bash
mpirun -np 2 python send-py-object.py
```

## 案例2：发送 NumPy `ndarray`

或者发送一个 NumPy `ndarray`，如下：

```python
from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

# 明确告知 MPI 数据类型为 int
# dtype='i', i 为 INT 的缩写
if rank == 0:
    data = np.arange(10, dtype='i')
    comm.Send([data, MPI.INT], dest=1)
    print(f"Sended: {data}, from rank: {rank}.")
elif rank == 1:
    data = np.empty(10, dtype='i')
    comm.Recv([data, MPI.INT], source=0)
    print(f"Received: {data}, to rank: {rank}.")

# MPI 自动发现数据类型
if rank == 0:
    data = np.arange(10, dtype=np.float64)
    comm.Send(data, dest=1)
    print(f"Sended: {data}, from rank: {rank}.")
elif rank == 1:
    data = np.empty(10, dtype=np.float64)
    comm.Recv(data, source=0)
    print(f"Received: {data}, to rank: {rank}.")
```

```{note}
这里的 `Send` 和 `Recv` 函数的首字母都大写了，因为大写的 `Send` 和 `Recv` 等方法是基于缓存（Buffer）的。对于这些基于缓存的函数，应该明确数据的类型，比如传入这样的二元组 `[data, MPI.DOUBLE]` 或三元组 `[data, count, MPI.DOUBLE]`。刚才例子中，`comm.Send(data, dest=1)` 没有明确告知 MPI 其数据类型和数据大小，是因为 MPI 对 NumPy 和 cupy `ndarray` 做了类型的自动探测。
```

## 案例3：Master-Worker

现在我们做一个 Master-Worker 案例，共有 `size` 个进程，前 `size-1` 个进程作为 Worker，随机生成数据，最后一个进程（Rank 为 `size-1`）作为 Master，接收数据，并将数据的大小打印出来。

```python
from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

if rank < size - 1:
    # Worker 进程
    np.random.seed(rank)
    # 随机生成
    data_count = np.random.randint(100)
    data = np.random.randint(100, size=data_count)
    comm.send(data, dest=size - 1)
    print(f"Worker: worker ID: {rank}; count: {len(data)}")
else:
    # Master 进程
    for i in range(size - 1):
        status = MPI.Status()
        data = comm.recv(source=MPI.ANY_SOURCE, status=status)
        print(f"Master: worker ID: {status.Get_source()}; count: {len(data)}")

comm.Barrier()
```

在这个例子中，`rank` 小于 `size - 1` 的进程是 Worker，随机生成数据，并发送出给最后一个进程（进程 Rank 号为 `size - 1`）。最后一个进程接收数据，并打印出接收数据的大小。

## 案例4：梯形法求PI值

对半径为R的四分之一圆，我们可以采用微分方法依次排列小矩形(类似于梯形台阶)，当矩形数量达到无穷大的N时,矩形总面积就接近于真实的1/4圆面积。假设此时有`size`个进程参与计算，首先求每个进程需要处理的矩形数量(`N/size`)。接下来，分配后`size-1`个进程作为Worker，计算各自矩形面积之和发送给主进程。而第一个进程作为Master，接收各Worker发送数据，汇总所有矩形面积，从而近似计算出PI值。

```python
from mpi4py import MPI
import math
communicator=MPI.COMM_WORLD
rank=communicator.Get_rank()  #进程唯一的标识Rank
process_nums=communicator.Get_size()
rect_num=64 * 1024 * 1024  #N
redius=1  #R
rect_width=redius/rect_num
step_size=rect_num//process_nums

def cal_rect_area(process_no,step_size,rect_width):
    tot_area=0.0
    rect_start=(process_no*step_size+1)*rect_width
    for i in range(step_size):
        x=rect_start+i*rect_width
        #  (x,y) 对应于第i个小矩形唯一在圆弧上的顶点，容易知道x^2+y^2=r^2  ==> y=sqrt(r^2-x^2)
        rect_length=math.pow(redius*redius-x*x, 0.5)
        tot_area+=rect_width*rect_length
    return tot_area

tot_area=cal_rect_area(rank,step_size,rect_width)
if rank==0:
    # Master 进程
    for i in range(1,process_nums):
        tot_area+=communicator.recv(source=i)
    p_i=tot_area * 4
    print('模拟PI值为: {:.10f}, 相对误差为：{:.10f}'.format(p_i,abs(1-p_i/math.pi))) 
else:
    # Worker 进程
    communicator.send(tot_area,dest=0)
```

上述例子中，我们设置常量参数为：`R=1`, `N=64*1024*1024`。启动程序需要在命令行中输入：

```bash
mpirun -np 8 python rectangle_approx_pi.py
```

## 阻塞 v.s. 非阻塞

### 阻塞

我们先分析一下阻塞式通信。`Send` 和 `Recv` 这两个基于缓存的方法：

* `Send` 直到缓存是空的时候，也就是说缓存中的数据都被发送出去后，才返回（`return`），允许运行用户代码中剩下的业务逻辑。缓存区域可以被接下来其他的 `Send` 循环再利用。
* `Recv` 直到缓存区域数据到达，才返回（`return`），，允许运行用户代码中剩下的业务逻辑。

如 {ref}`mpi-communications` 所示，阻塞通信是数据完成传输，才会返回（`return`），否则一直在等待。

```{figure} ../img/ch-mpi/blocking.svg
---
width: 800px
name: blocking-communications
---
阻塞式通信示意图
```

阻塞式通信的代码更容易去设计，但出现问题是死锁，比如类似下面的逻辑，Rank = 1 的产生了死锁，应该将 `Send` 和 `Recv` 调用顺序互换

```python
if rank == 0:
	comm.Send(..to rank 1..)
    comm.Recv(..from rank 1..)
else if (rank == 1): <- 该进程死锁
    comm.Send(..to rank 0..)       <- 应将 Send Revc 互换
    comm.Recv(..from rank 0..)
```

### 非阻塞

非阻塞式通信调用后直接返回 `Request` 句柄（Handle），程序员接下来再对 `Request` 做处理，比如等待 `Request` 涉及的数据传输完毕。非阻塞式通信有大写的 i（I） 作为前缀， `Irecv` 的函数参数与之前相差不大，只不过返回值是一个 `Request`：`Request = Isend(buf, dest, tag=0`。 `Request` 类提供了 `wait` 方法，显示地调用 `wait()` 可以等待数据传输完毕。用 `Isend` 写的阻塞式的代码，可以改为 `Isend` + `Request.wait()` 以非阻塞方式实现。

```python
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

if rank == 0:
    data = {'a': 7, 'b': 3.14}
    req = comm.isend(data, dest=1, tag=11)
    print(f"Sending: {data}, from rank: {rank}.")
    req.wait()
    print(f"Sended: {data}, from rank: {rank}.")
elif rank == 1:
    req = comm.irecv(source=0, tag=11)
    print(f"Receiving: to rank: {rank}.")
    data = req.wait()
    print(f"Received: {data}, to rank: {rank}.")
```

{numref}`non-blocking-communications` 展示非阻塞通信 `wait()` 加入后数据流的变化。

```{figure} ../img/ch-mpi/non-blocking.svg
---
width: 800px
name: non-blocking-communications
---
非阻塞式通信示意图
```

In [2]:
!mpirun -np 4 python broadcast.py

Rank: 0, data:[0. 1. 2. 3. 4.]
Rank: 2, data:[0. 1. 2. 3. 4.]
Rank: 1, data:[0. 1. 2. 3. 4.]
Rank: 3, data:[0. 1. 2. 3. 4.]


### Scatter 和 Gather

`Comm.Scatter` 和 `Comm.Gather` 是一组相对应的操作，如

`Comm.Scatter` 将数据从一个进程分散到组中的所有进程，一个进程将数据分散成多个块，每个块发送给对应的进程。其他进程接收并存储各自的块。Scatter 操作适用于将一个较大的数据集分割成多个小块。
`Comm.Gather` 与 `Comm.Scatter` 相反，将组里所有进程的小数据块归集到一个进程上。

```{figure} ../img/ch-mpi/scatter-gather.png
---
width: 600px
name: mpi-scatter-gather
---
Scatter 与 Gather
```

### 案例2：Scatter

```python
from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

sendbuf = None
if rank == 0:
    sendbuf = np.empty([size, 8], dtype='i')
    sendbuf.T[:,:] = range(size)
    print(f"Rank: {rank}, to be scattered: \n{sendbuf}")
recvbuf = np.empty(8, dtype='i')
comm.Scatter(sendbuf, recvbuf, root=0)
print(f"Rank: {rank}, after scatter: {recvbuf}")
assert np.allclose(recvbuf, rank)
```

In [3]:
!mpirun -np 4 python scatter.py

Rank: 0, to be scattered: 
[[0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1]
 [2 2 2 2 2 2 2 2]
 [3 3 3 3 3 3 3 3]]
Rank: 0, after scatter: [0 0 0 0 0 0 0 0]
Rank: 1, after scatter: [1 1 1 1 1 1 1 1]
Rank: 2, after scatter: [2 2 2 2 2 2 2 2]
Rank: 3, after scatter: [3 3 3 3 3 3 3 3]


### Allgather 和 Alltoall

另外两个比较复杂的操作是 `Comm.Allgather` 和 `Comm.Alltoall`。

`Comm.Allgather` 是 `Comm.Gather` 的进阶版，如 {numref}`mpi-allgather` 所示，它把散落在多个进程的多个小数据块发送给每个进程，每个进程都包含了一份相同的数据。

```{figure} ../img/ch-mpi/allgather.png
---
width: 600px
name: mpi-allgather
---
Allgather
```

`Comm.Alltoall` 是 `Comm.Scatter` 的 `Comm.Gather` 组合，如 {numref}`mpi-alltoall` 所示，先进行 `Comm.Scatter`，再进行 `Comm.Gather`。如果把数据看成一个矩阵，`Comm.Alltoall` 又可以被看做是一种全局的转置（Transpose）操作。

```{figure} ../img/ch-mpi/alltoall.png
---
width: 600px
name: mpi-alltoall
---
Alltoall
```

## 集合计算

集合计算指的是在将散落在不同进程的数据聚合在一起的时候，对数据进行计算，比如 `Comm.Reduce` 和 `Intracomm` 等。如 {numref}`mpi-reduce` 和 {numref}`mpi-scan` 所示，数据归集到某个进程时，还执行了聚合函数 `f`，常用的聚合函数有求和 `MPI.SUM` 等。

```{figure} ../img/ch-mpi/reduce.png
---
width: 600px
name: mpi-reduce
---
Reduce
```

```{figure} ../img/ch-mpi/scan.png
---
width: 600px
name: mpi-scan
---
Scan
```

In [ ]:
(mpi-point2point)=
# 点对点通信

一个最简单的通信模式点对点（Point-to-Point）通信，点对点通信又分为阻塞式（Blocking）和非阻塞式（Non-Blocking）。实现点对点时主要考虑两个问题：

* 如何控制和识别不同的进程？比如，想让 Rank 为 0 的进程给 Rank 为 1 的进程发消息。
* 如何控制数据的读写？多大的数据，数据类型是什么？

## 发送与接收

[`Comm.send`](https://mpi4py.readthedocs.io/en/latest/reference/mpi4py.MPI.Comm.html#mpi4py.MPI.Comm.send) 和 [`Comm.recv`](https://mpi4py.readthedocs.io/en/latest/reference/mpi4py.MPI.Comm.html#mpi4py.MPI.Comm.recv) 分别用来阻塞式地发送和接收数据。

`Comm.send(obj, dest, tag=0)` 的参数主要是 `obj` 和 `dest`。`obj` 就是我们想要发送的数据，数据可以是 Python 内置的数据类型，比如 `list` 和 `dict` 等，也可以是 NumPy 的 `ndarray`，甚至是 GPU 上的 cupy 数据。上一节 {ref}`mpi-hello-world` 我们介绍了 Communicator 和 Rank，可以通过 Rank 的号码来定位一个进程。`dest` 可以用 Rank 号码来表示。`tag` 主要用来标识，给程序员一个精细控制的选项，使用 `tag` 可以实现消息的有序传递和筛选。接收方可以选择只接收特定标签的消息，或者按照标签的顺序接收消息，以便更加灵活地控制消息的发送和接收过程。

## 案例1：发送 Python 对象

比如，我们发送一个 Python 对象。Python 对象在通信过程中的序列化使用的是 [pickle](https://docs.python.org/3/library/pickle.html#module-pickle)。

```python
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

if rank == 0:
    data = {'a': 7, 'b': 3.14}
    comm.send(data, dest=1)
    print(f"Sended: {data}, from rank: {rank}.")
elif rank == 1:
    data = comm.recv(source=0)
    print(f"Received: {data}, to rank: {rank}.")
```

在命令行中这样启动：

```bash
mpirun -np 2 python send-py-object.py
```

## 案例2：发送 NumPy `ndarray`

或者发送一个 NumPy `ndarray`，如下：

```python
from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

# 明确告知 MPI 数据类型为 int
# dtype='i', i 为 INT 的缩写
if rank == 0:
    data = np.arange(10, dtype='i')
    comm.Send([data, MPI.INT], dest=1)
    print(f"Sended: {data}, from rank: {rank}.")
elif rank == 1:
    data = np.empty(10, dtype='i')
    comm.Recv([data, MPI.INT], source=0)
    print(f"Received: {data}, to rank: {rank}.")

# MPI 自动发现数据类型
if rank == 0:
    data = np.arange(10, dtype=np.float64)
    comm.Send(data, dest=1)
    print(f"Sended: {data}, from rank: {rank}.")
elif rank == 1:
    data = np.empty(10, dtype=np.float64)
    comm.Recv(data, source=0)
    print(f"Received: {data}, to rank: {rank}.")
```

```{note}
这里的 `Send` 和 `Recv` 函数的首字母都大写了，因为大写的 `Send` 和 `Recv` 等方法是基于缓存（Buffer）的。对于这些基于缓存的函数，应该明确数据的类型，比如传入这样的二元组 `[data, MPI.DOUBLE]` 或三元组 `[data, count, MPI.DOUBLE]`。刚才例子中，`comm.Send(data, dest=1)` 没有明确告知 MPI 其数据类型和数据大小，是因为 MPI 对 NumPy 和 cupy `ndarray` 做了类型的自动探测。
```

## 案例3：Master-Worker

现在我们做一个 Master-Worker 案例，共有 `size` 个进程，前 `size-1` 个进程作为 Worker，随机生成数据，最后一个进程（Rank 为 `size-1`）作为 Master，接收数据，并将数据的大小打印出来。

```python
from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

if rank < size - 1:
    # Worker 进程
    np.random.seed(rank)
    # 随机生成
    data_count = np.random.randint(100)
    data = np.random.randint(100, size=data_count)
    comm.send(data, dest=size - 1)
    print(f"Worker: worker ID: {rank}; count: {len(data)}")
else:
    # Master 进程
    for i in range(size - 1):
        status = MPI.Status()
        data = comm.recv(source=MPI.ANY_SOURCE, status=status)
        print(f"Master: worker ID: {status.Get_source()}; count: {len(data)}")

comm.Barrier()
```

在这个例子中，`rank` 小于 `size - 1` 的进程是 Worker，随机生成数据，并发送出给最后一个进程（进程 Rank 号为 `size - 1`）。最后一个进程接收数据，并打印出接收数据的大小。

## 阻塞 v.s. 非阻塞

### 阻塞

我们先分析一下阻塞式通信。`Send` 和 `Recv` 这两个基于缓存的方法：

* `Send` 直到缓存是空的时候，也就是说缓存中的数据都被发送出去后，才返回（`return`），允许运行用户代码中剩下的业务逻辑。缓存区域可以被接下来其他的 `Send` 循环再利用。
* `Recv` 直到缓存区域数据到达，才返回（`return`），，允许运行用户代码中剩下的业务逻辑。

如 {ref}`mpi-communications` 所示，阻塞通信是数据完成传输，才会返回（`return`），否则一直在等待。

```{figure} ../img/ch-mpi/blocking.svg
---
width: 800px
name: blocking-communications
---
阻塞式通信示意图
```

阻塞式通信的代码更容易去设计，但出现问题是死锁，比如类似下面的逻辑，Rank = 1 的产生了死锁，应该将 `Send` 和 `Recv` 调用顺序互换

```python
if rank == 0:
	comm.Send(..to rank 1..)
    comm.Recv(..from rank 1..)
else if (rank == 1): <- 该进程死锁
    comm.Send(..to rank 0..)       <- 应将 Send Revc 互换
    comm.Recv(..from rank 0..)
```

### 非阻塞

非阻塞式通信调用后直接返回 `Request` 句柄（Handle），程序员接下来再对 `Request` 做处理，比如等待 `Request` 涉及的数据传输完毕。非阻塞式通信有大写的 i（I） 作为前缀， `Irecv` 的函数参数与之前相差不大，只不过返回值是一个 `Request`：`Request = Isend(buf, dest, tag=0`。 `Request` 类提供了 `wait` 方法，显示地调用 `wait()` 可以等待数据传输完毕。用 `Isend` 写的阻塞式的代码，可以改为 `Isend` + `Request.wait()` 以非阻塞方式实现。

```python
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

if rank == 0:
    data = {'a': 7, 'b': 3.14}
    req = comm.isend(data, dest=1, tag=11)
    print(f"Sending: {data}, from rank: {rank}.")
    req.wait()
    print(f"Sended: {data}, from rank: {rank}.")
elif rank == 1:
    req = comm.irecv(source=0, tag=11)
    print(f"Receiving: to rank: {rank}.")
    data = req.wait()
    print(f"Received: {data}, to rank: {rank}.")
```

{numref}`non-blocking-communications` 展示非阻塞通信 `wait()` 加入后数据流的变化。

```{figure} ../img/ch-mpi/non-blocking.svg
---
width: 800px
name: non-blocking-communications
---
非阻塞式通信示意图
```